<table style="border:none; border-collapse:collapse; cellspacing:0; cellpadding:0">
<tr>
    <td width=30% style="border:none">
        <center>
            <img src="../images/iapau_icon.png" width="30%"/><br>
            <a href="https://iapau.org/">Association IA Pau</a><br>
            <a href="https://iapau.org/events/festival/">Festival IAPau 7</a>
        </center>
    </td>
    <td style="border:none">
        <center>
            <h1>Atelier - Agentic RAG</h1>
            <h2>Évaluation</h2>
        </center>
    </td>
    <td width=20% style="border:none">
    </td>
</tr>
</table>

---

**Prérequis :** Compléter les Phases 0-3.

---

Nous évaluerons notre système argentique selon trois dimensions; quantitative, qualitative et performance.

## 📋 Table des Matières

1. [**Évaluation quantitative (Qualité de récupération)**](#quantitative-evaluation)
   - Test de performance de l'outil _Librarian_
   - Mesure de la précision et du rappel

2. [**Évaluation qualitative (LLM-as-a-Judge)**](#qualitative-evaluation)
   - Évaluation structurée avec un juge
   - Analyse de la fidélité et pertinence des réponses

3. [**Évaluation des performances (Vitesse et coût)**](#performance-evaluation)
   - Mesure de la latence end-to-end
   - Analyse des coûts opérationnels

---

In [25]:
import os
import json
import time
from typing import List, Dict

from langchain_openai import ChatOpenAI
from langchain_core.callbacks.base import BaseCallbackHandler
from pydantic import BaseModel, Field

from specialist_tools import (
    librarian_rag_tool,
    analyst_sql_tool,
    analyst_trend_tool,
    tools,
    tool_map
)

### <b><div style='padding:15px;background-color:#4A5568;color:white;border-radius:2px;font-size:110%;text-align: left'>1. Évaluation quantitative</div></b>

<div style='display:flex;align-items:center;gap:16px;background:linear-gradient(90deg,#f0f7ff,#ffffff);padding:16px;border-left:6px solid #2b6cb0;border-radius:8px;'>
  <div style='width:5%;min-width:64px;text-align:center;font-size:44px;line-height:1;'>🚀</div>
  <div style='width:90%;color:#000;'>
    <h2 style='margin:0 0 6px 0;color:#000;font-size:1.15em;'>Ce que nous allons faire</h2>
    <p style='margin:0 0 8px 0;color:#000;'>Une excellente réponse est impossible sans une excellente récupération des données pertinentes. Nous devons mesurer la performance de notre outil <i>Librarian</i>. Nous créerons un petit ensemble de test de questions et identifierons manuellement les chunks sources qui devraient être récupérés.</p>
    <p style='margin:0 8px 0 0;font-weight:600;color:#000;'>Ensuite nous mesurerons :</p>
    <ul style='margin:8px 0 0 18px;'>
      <li><strong>Précision du Contexte</strong> : Parmi les chunks que le <i>Librarian</i> a récupérés, combien étaient réellement pertinents ?</li>
      <li><strong>Rappel du Contexte</strong> : Parmi tous les chunks pertinents possibles, combien le Librarian en a-t-il trouvés ?</li>
    </ul>
  </div>
</div>

In [26]:
# NOTE: Dans un projet réel, l'identification des golden_doc_ids est un processus manuel qui prend beaucoup de temps.
# Ici, nous allons le simuler en exécutant d'abord le récupérateur, puis en sélectionnant les ID pertinents dans son résultat.

eval_questions = [
    "What were the key drivers of revenue for the IA plateform segment?",
    "Describe the company's strategy regarding Artificial Intelligence.",
    "What are the material legal proceedings the company is involved in?"
]

# For this demo, we'll run the retriever and then pretend we manually picked the IDs.
# This simulates having a ground truth dataset.
ground_truth = {}
for q in eval_questions:
    results = librarian_rag_tool.invoke(q)
    # Simulate a human choosing the truly relevant docs from the top results
    ground_truth[q] = [res['content'] for i, res in enumerate(results) if i < 3] # Pick top 3 as golden

def evaluate_retrieval(question: str, retrieved_docs: List[Dict]) -> Dict[str, float]:
    golden_docs = ground_truth[question]
    retrieved_contents = [doc['content'] for doc in retrieved_docs]
    
    # True positives: docs that are in both retrieved and golden sets
    tp = len(set(retrieved_contents) & set(golden_docs))
    
    precision = tp / len(retrieved_contents) if retrieved_contents else 0
    recall = tp / len(golden_docs) if golden_docs else 0
    
    return {"precision": precision, "recall": recall}

print("--- Retrieval Quality Evaluation ---")
all_metrics = []
for q in eval_questions:
    retrieved = librarian_rag_tool.invoke(q)
    metrics = evaluate_retrieval(q, retrieved)
    all_metrics.append(metrics)
    print(f"\nQuestion: {q}")
    print(f"  - Precision: {metrics['precision']:.2f}")
    print(f"  - Recall: {metrics['recall']:.2f}")

# Calculate average scores
avg_precision = sum(m['precision'] for m in all_metrics) / len(all_metrics)
avg_recall = sum(m['recall'] for m in all_metrics) / len(all_metrics)

print(f"\n\nAverage Precision: {avg_precision:.2f}")
print(f"Average Recall: {avg_recall:.2f}")


-- Librarian Tool Called with query: 'What were the key drivers of revenue for the IA plateform segment?' --
  - Optimized query: 'Key drivers of revenue for the IA platform segment in corporate financial reports'
  - Retrieved 20 candidate chunks
  - Re-ranked results
  - Returning top 5 chunks

-- Librarian Tool Called with query: 'Describe the company's strategy regarding Artificial Intelligence.' --
  - Optimized query: 'Analyze the company's strategic initiatives and investments in Artificial Intelligence as outlined in financial reports and corporate strategy documents.'
  - Retrieved 20 candidate chunks
  - Re-ranked results
  - Returning top 5 chunks

-- Librarian Tool Called with query: 'What are the material legal proceedings the company is involved in?' --
  - Optimized query: 'List all ongoing material legal proceedings involving the company, including case names, jurisdictions, and potential financial impacts.'
  - Retrieved 20 candidate chunks
  - Re-ranked results
  - R

<div style='display:flex;align-items:center;gap:16px;background:linear-gradient(90deg,#f0fff4,#ffffff);padding:16px;border-left:6px solid #16a34a;border-radius:8px;'>
  <div style='width:5%;min-width:64px;text-align:center;font-size:44px;line-height:1;'>✅</div>
  <div style='width:90%;color:#000;'>
    <h2 style='margin:0 0 6px 0;color:#000;font-size:1.15em;'>Discussion du Résultat</h2>
    <p style='margin:0 0 8px 0;color:#000;'>Ce résultat nous donne nos premiers chiffres concrets sur la performance d'un composant clé.</p> 
    <p style='margin:0 0 8px 0;color:#000;'>Une précision élevée signifie que les résultats obtenus par notre agent sont pertinents et non remplis de bruit.</p> 
    <p style='margin:0 0 8px 0;color:#000;'>Un rappel élevé signifie que nous ne manquons pas d'informations importantes.</p> 
    <p style='margin:0 0 8px 0;color:#000;'>Les résultats ici sont assez bons, ce qui témoigne de notre stratégie RAG avancée (optimisation de la requête + re-classement). Dans un vrai projet, vous étendriez cet ensemble d'évaluation et suivriez ces métriques au fil du temps en apportant des modifications au système.</p>
  </div>
</div>

### <b><div style='padding:15px;background-color:#4A5568;color:white;border-radius:2px;font-size:110%;text-align: left'>2. Évaluation qualitative (LLM-as-a-Judge)</div></b>

<div style='display:flex;align-items:center;gap:16px;background:linear-gradient(90deg,#f0f7ff,#ffffff);padding:16px;border-left:6px solid #2b6cb0;border-radius:8px;'>
  <div style='width:5%;min-width:64px;text-align:center;font-size:44px;line-height:1;'>🚀</div>
  <div style='width:90%;color:#000;'>
    <h2 style='margin:0 0 6px 0;color:#000;font-size:1.15em;'>Ce que nous allons faire</h2>
    <p style='margin:0 0 8px 0;color:#000;'>Pour un moteur de raisonnement, la justesse est nécessaire mais pas suffisante. Nous devons mesurer la <strong>qualité</strong> de ses analyses.</p>
    <ul style='margin:8px 0 0 18px;'>
      <li><strong>Fidélité</strong> : La réponse contredit-elle les sources fournies ? (score 1-5)</li>
      <li><strong>Pertinence de la Réponse</strong> : La réponse répond-elle directement à la question de l'utilisateur ? (score 1-5)</li>
      <li><strong>Solidité du Plan</strong> : Le plan de l'agent était-il logique et efficace ? (score 1-5)</li>
      <li><strong>Profondeur Analytique</strong> : L'agent a-t-il dépassé la synthèse des faits pour générer une hypothèse ou analyse précieuse et fondée sur les données ? (score 1-5)</li>
    </ul>
  </div>
</div>

In [27]:
import os

class AdvancedEvaluationResult(BaseModel):
    """Structured output for the advanced LLM-as-a-Judge evaluation."""
    faithfulness_score: int = Field(description="Score from 1-5 for faithfulness.")
    relevance_score: int = Field(description="Score from 1-5 for answer relevance.")
    plan_soundness_score: int = Field(description="Score from 1-5 for the plan's logic and efficiency.")
    analytical_depth_score: int = Field(description="Score from 1-5 for generating insightful, data-grounded hypotheses.")
    reasoning: str = Field(description="Detailed step-by-step reasoning for all scores.")

# OpenAI LLM for judging - GPT-4o
judge_llm = ChatOpenAI(
    model="gpt-4o", api_key=os.getenv('OPENAI_API_KEY'), temperature=0).with_structured_output(AdvancedEvaluationResult)

def get_advanced_judge_prompt(request, plan, context, answer):
    return f"""You are an impartial AI evaluator. Your task is to rigorously evaluate the performance of a financial analyst AI agent based on the provided information and a strict rubric.
**The User's Request:**\n{request}\n
**The Agent's Plan:**\n{plan}\n
**The Context Used by the Agent (Source Data):**\n{context}\n
**The Agent's Final Answer:**\n{answer}\n
---\n**Evaluation Rubric:**\n
1.  **Faithfulness (1-5):** Is the answer entirely supported by the provided context?\n
2.  **Answer Relevance (1-5):** Does the answer perfectly and comprehensively respond to the user's request?\n
3.  **Plan Soundness (1-5):** Was the agent's plan optimal - the most logical and efficient way to answer the request?\n
4.  **Analytical Depth (1-5):** Did the agent generate a valuable, data-grounded hypothesis that connects disparate facts, or did it just list information? (1=Lists facts, 3=Makes a simple connection, 5=Generates a novel, insightful, and well-supported hypothesis).\n
Please provide your scores and detailed reasoning.\n"""

def evaluate_with_advanced_judge(request: str, full_graph_output: Dict) -> AdvancedEvaluationResult:
    plan = full_graph_output.get('plan', [])
    context = full_graph_output.get('intermediate_steps', [])
    answer = full_graph_output.get('final_response', '')
    prompt = get_advanced_judge_prompt(request, plan, context, answer)
    return judge_llm.invoke(prompt)

# --- Initialize the orchestrator and run a complex query ---
from agent_orchestrator import FinancialAgentOrchestrator

print("Initializing Agent Orchestrator...")
orchestrator = FinancialAgentOrchestrator()

# Run a complex query to get the state
complex_query = "What are the key AI risks mentioned in the 10-K filing and how do they relate to revenue trends in the Intelligent Cloud segment?"
print(f"\nRunning complex query: {complex_query}")
complex_run_state = orchestrator.run(complex_query)

# Now evaluate
judge_evaluation = evaluate_with_advanced_judge(complex_run_state['original_request'], complex_run_state)

print("\n--- LLM-as-a-Judge Evaluation Result ---")
print(json.dumps(judge_evaluation.model_dump(), indent=2))

Initializing Agent Orchestrator...
✓ Graph compiled successfully!
✓ Agent orchestrator initialized successfully!

Running complex query: What are the key AI risks mentioned in the 10-K filing and how do they relate to revenue trends in the Intelligent Cloud segment?

🚀 RUNNING AGENT ORCHESTRATOR
📝 Query: What are the key AI risks mentioned in the 10-K filing and how do they relate to revenue trends in the Intelligent Cloud segment?


-- Gatekeeper (Ambiguity Check) Node --
  - Request is specific. Proceeding to planner.

-- Planner Node --
  - Raw plan response: ["librarian_rag_tool('key AI risks mentioned in NVIDIA 10-K filing')", "librarian_rag_tool('Intelligent Cloud segment revenue and risk discussion in 10-K')", "analyst_trend_tool('analyze revenue trend in Intelligent Cloud segment')", "FINISH"]
  - Generated Plan: ["librarian_rag_tool('key AI risks mentioned in NVIDIA 10-K filing')", "librarian_rag_tool('Intelligent Cloud segment revenue and risk discussion in 10-K')", "analyst_

<div style='display:flex;align-items:center;gap:16px;background:linear-gradient(90deg,#f0fff4,#ffffff);padding:16px;border-left:6px solid #16a34a;border-radius:8px;'>
  <div style='width:5%;min-width:64px;text-align:center;font-size:44px;line-height:1;'>✅</div>
  <div style='width:90%;color:#000;'>
    <h2 style='margin:0 0 6px 0;color:#000;font-size:1.15em;'>Discussion du Résultat</h2>
    <p style='margin:0 0 8px 0;color:#000;'>Les résultats d'évaluation confirment le succès de nos améliorations. L'agent reçoit des scores parfaits sur toute la ligne, mais le résultat le plus important est le 5/5 pour la Profondeur Analytique. Le raisonnement du Juge souligne explicitement la capacité de l'agent à synthétiser une <i>hypothèse cohérente</i> comme un acte analytique de grande valeur. Ce feedback qualitatif automatisé fournit une preuve solide que notre agent opère à un niveau d'abstraction élevé. Il ne fait pas que récupérer et résumer ; il analyse et infère.</p>
  </div>
</div>

### <b><div style='padding:15px;background-color:#4A5568;color:white;border-radius:2px;font-size:110%;text-align: left'>3. Évaluation des performances (Vitesse et coût)</div></b>

<div style='display:flex;align-items:center;gap:16px;background:linear-gradient(90deg,#f0f7ff,#ffffff);padding:16px;border-left:6px solid #2b6cb0;border-radius:8px;'>
  <div style='width:5%;min-width:64px;text-align:center;font-size:44px;line-height:1;'>🚀</div>
  <div style='width:90%;color:#000;'>
    <h2 style='margin:0 0 6px 0;color:#000;font-size:1.15em;'>Ce que nous allons faire</h2>
    <p style='margin:0 0 8px 0;color:#000;'>Dans le monde réel, la qualité ne suffit pas. Un agent trop lent ou trop coûteux n'est pas pratique. Nous mesurerons deux métriques de performance clés :</p>
    <ul style='margin:8px 0 0 18px;'>
      <li><strong>Latence End-to-End</strong> : Combien de temps prend une requête du début à la fin ?</li><br>
      <li><strong>Coût par requête</strong> : Nous estimerons le coût basé sur l'utilisation de tokens de nos LLMs.</li>
    </ul>
  </div>
</div>

In [28]:
import time
from langchain_core.callbacks.base import BaseCallbackHandler

class TokenCostCallback(BaseCallbackHandler):
    """A callback to track token usage and estimate cost."""
    def __init__(self):
        super().__init__()
        self.total_prompt_tokens = 0
        self.total_completion_tokens = 0
        # Pricing GPT-4o - Novembre 2025 (USD / 1M tokens)
        self.prompt_cost_per_1m = 2.50
        self.completion_cost_per_1m = 10.00

    def on_llm_end(self, response, **_kwargs):
        usage = response.llm_output.get('token_usage', {})
        self.total_prompt_tokens += usage.get('prompt_tokens', 0)
        self.total_completion_tokens += usage.get('completion_tokens', 0)
        
    def get_summary(self):
        prompt_cost = (self.total_prompt_tokens / 1_000_000) * self.prompt_cost_per_1m
        completion_cost = (self.total_completion_tokens / 1_000_000) * self.completion_cost_per_1m
        total_cost = prompt_cost + completion_cost
        return {
            "total_prompt_tokens": self.total_prompt_tokens,
            "total_completion_tokens": self.total_completion_tokens,
            "estimated_cost_usd": total_cost
        }

In [31]:
# --- Run a query with performance tracking ---
cost_tracker = TokenCostCallback()
start_time = time.time()

# Utilisez l'application Orchestrator
orchestrator.app.invoke(
    {"original_request": "Analyze Nvidia's revenue trend from 2020 to 2024 and discuss the risk strategy, mentioned in their reports, to increase these revenues."},
    config={'callbacks': [cost_tracker]}
)

end_time = time.time()
latency = end_time - start_time
cost_summary = cost_tracker.get_summary()

print("--- Performance Evaluation ---")
print(f"End-to-End Latency: {latency:.2f} seconds")
print("\nCost Summary:")
print(json.dumps(cost_summary, indent=2))


-- Gatekeeper (Ambiguity Check) Node --
  - Request is specific. Proceeding to planner.

-- Planner Node --
  - Raw plan response: ["analyst_trend_tool('analyze Nvidia revenue trend from 2020 to 2024')", "librarian_rag_tool('risk strategy to increase revenue as mentioned in Nvidia financial reports')", "FINISH"]
  - Generated Plan: ["analyst_trend_tool('analyze Nvidia revenue trend from 2020 to 2024')", "librarian_rag_tool('risk strategy to increase revenue as mentioned in Nvidia financial reports')", 'FINISH']

-- Tool Executor Node --
  - Executing tool: analyst_trend_tool with input: 'analyze Nvidia revenue trend from 2020 to 2024'

-- Analyst Trend Tool Called with query: 'analyze Nvidia revenue trend from 2020 to 2024' --

-- Auditor (Self-Correction) Node --
  - Audit Confidence Score: 4/5

-- Advanced Router Node --
  - Decision: Plan has more steps. Routing to tool executor.

-- Tool Executor Node --
  - Executing tool: librarian_rag_tool with input: 'risk strategy to increase

<div style='display:flex;align-items:center;gap:16px;background:linear-gradient(90deg,#f0fff4,#ffffff);padding:16px;border-left:6px solid #16a34a;border-radius:8px;'>
  <div style='width:5%;min-width:64px;text-align:center;font-size:44px;line-height:1;'>✅</div>
  <div style='width:90%;color:#000;'>
    <h2 style='margin:0 0 6px 0;color:#000;font-size:1.15em;'>Discussion du Résultat</h2>
    <p style='margin:0 0 8px 0;color:#000;'>Cette évaluation des performances nous donne des données opérationnelles critiques.</p> 
    <p style='margin:0 0 8px 0;color:#000;'>Nous voyons que la latence pour une requête complexe est légèrement plus longue que pour une requête plus simple, ce qui est attendu étant donné les étapes cognitives supplémentaires (audit, synthèse plus complexe).<p> 
    <p style='margin:0 0 8px 0;color:#000;'>Le coût par requête est également un facteur dû aux multiples appels LLM dans les différents outils/agents. Ces métriques sont cruciales pour un déploiement réel. Elles montrent un compromis clair : pour gagner en profondeur analytique et en fiabilité, le coût augmente significativement.</p>
  </div>
</div>